In [ ]:
from google.colab import files
uploaded = files.upload()
uploaded = files.upload()

Saving merged_dataset.csv to merged_dataset.csv


Saving test.csv to test.csv


In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

In [ ]:
df = pd.read_csv('merged_dataset.csv')
df.head()

,class,conversation
0,일반대화,이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...
1,일반대화,회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워.\n큰 프로젝트를...
2,일반대화,상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워.\n직장 상사가 무섭게 생기셔서...
3,일반대화,이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다.\n첫 직...
4,일반대화,직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼.\n직장 사람들 관계에 대해서 고...


In [ ]:
texts = df['conversation'].astype(str).tolist()
labels = df['class'].astype(str).tolist()

In [ ]:
le = LabelEncoder()
y = le.fit_transform(labels)
num_classes = len(le.classes_)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    texts, y, test_size=0.2, stratify=y, random_state=42
)

In [ ]:
MAX_LEN = 512

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

VOCAB_SIZE = 20000
tokenizer_ks = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer_ks.fit_on_texts(X_train)

train_seq = tokenizer_ks.texts_to_sequences(X_train)
test_seq  = tokenizer_ks.texts_to_sequences(X_test)
train_seq = pad_sequences(train_seq, maxlen=MAX_LEN, padding="post", truncating="post")
test_seq  = pad_sequences(test_seq,  maxlen=MAX_LEN, padding="post", truncating="post")

In [ ]:

TOKENIZER_NAME = "monologg/kobert"
tokenizer = BertTokenizer.from_pretrained(TOKENIZER_NAME)

def encode_bert(texts):
    enc = tokenizer(
        texts,
        max_length=MAX_LEN,
        padding="max_length",
        truncation=True,
        return_tensors="tf"
    )
    return enc["input_ids"], enc["attention_mask"]

train_ids, train_mask = encode_bert(X_train)
test_ids,  test_mask  = encode_bert(X_test)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [ ]:
# 4. KoBERT 서브클래싱 모델 정의
class KoBertClassifier(tf.keras.Model):
    def __init__(self, num_classes):
        super().__init__()
        self.bert = TFBertModel.from_pretrained(TOKENIZER_NAME)
        self.dense1 = tf.keras.layers.Dense(128, activation="relu")
        self.drop   = tf.keras.layers.Dropout(0.3)
        self.out    = tf.keras.layers.Dense(num_classes, activation="softmax")

    def call(self, inputs, training=False):
        # inputs: dict with "input_ids" and "attention_mask"
        bert_out = self.bert(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            training=training
        )
        cls_tok = bert_out.last_hidden_state[:,0,:]      # [CLS] 토큰 임베딩
        x = self.dense1(cls_tok)
        x = self.drop(x, training=training)
        return self.out(x)

In [ ]:
model_bert = KoBertClassifier(num_classes)
model_bert.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:

history_bert = model_bert.fit(
    {"input_ids": train_ids, "attention_mask": train_mask},
    y_train,
    validation_split=0.1,
    epochs=3,
    batch_size=4,
    verbose=1
)

Epoch 1/3
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 303s 198ms/step - accuracy: 0.4281 - loss: 1.4079 - val_accuracy: 0.5172 - val_loss: 1.0895
Epoch 2/3
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 293s 186ms/step - accuracy: 0.5674 - loss: 1.0894 - val_accuracy: 0.5859 - val_loss: 0.9707
Epoch 3/3
1375/1375 ━━━━━━━━━━━━━━━━━━━━ 261s 185ms/step - accuracy: 0.5905 - loss: 1.0219 - val_accuracy: 0.6088 - val_loss: 0.9319


In [ ]:
# 6. BiLSTM 모델 정의 & 학습
from tf_keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense
from tensorflow.keras import Input, Model

seq_input = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32)  # 여기만 변경

x = Embedding(input_dim=VOCAB_SIZE, output_dim=100, input_length=MAX_LEN)(seq_input)
x = Bidirectional(LSTM(128, return_sequences=False))(x)
x = Dropout(0.3)(x)
out = Dense(num_classes, activation="softmax")(x)

earlystop = EarlyStopping(
    monitor="val_loss",
    patience=1,
    restore_best_weights=True
)

model_lstm = tf.keras.Model(seq_input, out)
model_lstm.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


history_lstm = model_lstm.fit(
    train_seq,
    y_train,
    validation_split=0.1,
    epochs=10,       # 충분히 크게
    batch_size=16,
    callbacks=[earlystop],
    verbose=1
)

Epoch 1/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 19s 41ms/step - accuracy: 0.5837 - loss: 0.9911 - val_accuracy: 0.7643 - val_loss: 0.6073
Epoch 2/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 18s 41ms/step - accuracy: 0.8608 - loss: 0.4053 - val_accuracy: 0.8691 - val_loss: 0.3724
Epoch 3/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 21s 41ms/step - accuracy: 0.9672 - loss: 0.1216 - val_accuracy: 0.8625 - val_loss: 0.3966


In [ ]:
probs_bert = model_bert.predict(
    {"input_ids": test_ids, "attention_mask": test_mask},
    batch_size=16
)
probs_lstm = model_lstm.predict(test_seq, batch_size=32)

w_bert, w_lstm = 0.3, 0.7
final_probs   = w_bert * probs_bert + w_lstm * probs_lstm
final_preds   = np.argmax(final_probs, axis=1)

# 8. 평가
f1 = f1_score(y_test, final_preds, average="macro")
print(f"Ensemble Macro-F1: {f1:.4f}")

96/96 ━━━━━━━━━━━━━━━━━━━━ 48s 500ms/step
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
Ensemble Macro-F1: 0.7853


In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# ——— 1. 테스트 파일 로드 (레이블 컬럼 없음) ———

df_test = pd.read_csv("test.csv")
texts_test = df_test["text"].astype(str).tolist()

# ——— 2. 입력 전처리: KoBERT & BiLSTM ———
# (encode_bert, keras_tokenizer, MAX_LEN 은 앞에서 정의한 함수/변수 그대로 사용)
test_ids, test_mask = encode_bert(texts_test)

raw_test_seq = tokenizer_ks.texts_to_sequences(texts_test)
test_seq     = pad_sequences(
    raw_test_seq,
    maxlen=MAX_LEN,
    padding="post",
    truncating="post"
)

# ——— 3. 모델별 예측 확률 ———
probs_bert_test = model_bert.predict(
    {"input_ids": test_ids, "attention_mask": test_mask},
    batch_size=4,
    verbose=1
)
probs_lstm_test = model_lstm.predict(
    test_seq,
    batch_size=16,
    verbose=1
)

# ——— 4. 앙상블 & 클래스 추출 ———
w_bert, w_lstm = 0.1, 0.9    # 검증에서 찾은 최적 가중치
probs_ensemble = w_bert * probs_bert_test + w_lstm * probs_lstm_test
pred_ids       = np.argmax(probs_ensemble, axis=1)

# LabelEncoder로 원래 문자열 레이블 복원
pred_labels = le.inverse_transform(pred_ids)

# 6. 문자열 레이블 복원 & 저장
pred_labels = le.inverse_transform(pred_ids)
df_test["predicted_label"] = pred_labels

# 변경된 저장 경로: Colab에서는 /content 아래에 저장
OUT_PATH = "/content/test_predictions.csv"
df_test.to_csv(OUT_PATH, index=False)
print(f"✅ 예측 완료! 결과를 '{OUT_PATH}'에 저장했습니다.")


125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 134ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
✅ 예측 완료! 결과를 '/content/test_predictions.csv'에 저장했습니다.


In [ ]:
df = pd.read_csv("test_predictions.csv")

# 3) 클래스별 분포 계산 및 출력
dist = df['predicted_label'].value_counts().sort_index()
print("=== Class Distribution ===")
print(dist)

# 4) 필요하다면 DataFrame으로 보기
dist_df = dist.reset_index()
dist_df.columns = ['predicted_label', 'count']
print("\n=== Distribution as DataFrame ===")
print(dist_df)

=== Class Distribution ===
predicted_label
갈취 대화          139
기타 괴롭힘 대화      109
일반대화            17
직장 내 괴롭힘 대화    149
협박 대화           86
Name: count, dtype: int64

=== Distribution as DataFrame ===
  predicted_label  count
0           갈취 대화    139
1       기타 괴롭힘 대화    109
2            일반대화     17
3     직장 내 괴롭힘 대화    149
4           협박 대화     86


In [ ]:

uploaded = files.upload()



Saving submission.csv to submission.csv


In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1) Kaggle 제공 test.csv 불러오기
TEST_PATH = "/content/test.csv"
test_df   = pd.read_csv(TEST_PATH)
idxs      = test_df["idx"].tolist()
texts     = test_df["text"].astype(str).tolist()

# 2) KoBERT 입력 생성
input_ids, attention_masks = encode_bert(texts)

# 3) BiLSTM 입력 생성
raw_seq = tokenizer_ks.texts_to_sequences(texts)
seq     = pad_sequences(
    raw_seq,
    maxlen=MAX_LEN,
    padding="post",
    truncating="post"
)

# 4) 모델별 예측 확률
probs_bert = model_bert.predict(
    {"input_ids": input_ids, "attention_mask": attention_masks},
    batch_size=16, verbose=1
)
probs_lstm = model_lstm.predict(seq, batch_size=32, verbose=1)

# 5) 단순 가중 평균 앙상블
probs_ens = w_bert * probs_bert + w_lstm * probs_lstm
pred_ids  = np.argmax(probs_ens, axis=1)  # 0,1,2,3,4

# 6) submission DataFrame 생성 (오직 idx, target)
submission = pd.DataFrame({
    "idx":    idxs,
    "target": pred_ids
})

# 7) CSV로 저장
OUT_PATH = "/content/final_submission.csv"
submission.to_csv(OUT_PATH, index=False)
print(f"✅ Kaggle 제출용 파일 생성 완료: {OUT_PATH}")


32/32 ━━━━━━━━━━━━━━━━━━━━ 17s 517ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
✅ Kaggle 제출용 파일 생성 완료: /content/final_submission.csv


In [ ]:
import pandas as pd

# 1) 기존 submission 불러오기 (idx, text, target 포함)
submission = pd.read_csv("/content/submission.csv")

# 2) text 컬럼 삭제
submission = submission.drop(columns=["text"])

# 3) target 업데이트 (pred_ids 는 이미 계산된 예측 인덱스 리스트)
submission["target"] = pred_ids

# 4) 최종 저장
submission.to_csv("/content/final_submission.csv", index=False)
print("✅ final_submission.csv 생성 완료!")


✅ final_submission.csv 생성 완료!


In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 0) 클래스 매핑 딕셔너리 정의
class_map = {
    "협박":           0,
    "갈취":           1,
    "직장 내 괴롭힘": 2,
    "기타 괴롭힘":    3,
    "일반대화":        4,
    "기타 괴롭힘 대화": 3,
    "직장 내 괴롭힘 대화": 3,
    "갈취 대화":           1,
    "협박 대화":           0,
}

# 1) Kaggle 제공 test.csv 불러오기
TEST_PATH = "/content/test.csv"
test_df   = pd.read_csv(TEST_PATH)
idxs      = test_df["idx"].tolist()
texts     = test_df["text"].astype(str).tolist()

# 2) KoBERT 입력 생성
input_ids, attention_masks = encode_bert(texts)

# 3) BiLSTM 입력 생성
raw_seq = tokenizer_ks.texts_to_sequences(texts)
seq     = pad_sequences(
    raw_seq,
    maxlen=MAX_LEN,
    padding="post",
    truncating="post"
)

# 4) 모델별 예측 확률
probs_bert = model_bert.predict(
    {"input_ids": input_ids, "attention_mask": attention_masks},
    batch_size=16, verbose=1
)
probs_lstm = model_lstm.predict(seq, batch_size=32, verbose=1)

# 5) 단순 가중 평균 앙상블
probs_ens = w_bert * probs_bert + w_lstm * probs_lstm
pred_ids  = np.argmax(probs_ens, axis=1)         # 0~4 (숫자 인덱스)

# 6) (선택) 숫자 인덱스를 다시 문자열 레이블로 변환하고 싶다면
pred_labels = le.inverse_transform(pred_ids)    # e.g. ["협박","갈취",...]

# 7) 문자열 레이블 → 숫자 코드로 확실히 매핑
pred_numeric = [ class_map[label] for label in pred_labels ]

# 8) 최종 Submission DataFrame 생성
submission = pd.DataFrame({
    "idx":    idxs,
    "target": pred_numeric
})

# 9) CSV로 저장
OUT_PATH = "/content/mapping_submission.csv"
submission.to_csv(OUT_PATH, index=False)
print("✅ 완전 일치하는 submission 파일 생성 완료:", OUT_PATH)


32/32 ━━━━━━━━━━━━━━━━━━━━ 16s 488ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
✅ 완전 일치하는 submission 파일 생성 완료: /content/mapping_submission.csv
